In [ ]:
from Crypto.Hash import SHA256

actions = [b'block',b'unblock',b'quarantine',b'unquarantine',b'host restart',b'wazuh restart']

action_key = {hash.decode(): SHA256.new(hash).hexdigest() for hash in actions}
action_key

In [ ]:
orgID = [b'root',b'OILK0',b'OV8LG',b'OATOG']
orgName = ['root','Eris','Nbs','Nico']

velociraptor_orgs = {orgName[n]: SHA256.new(orgID[n]).hexdigest() for n in range(len(orgID))}
velociraptor_orgs

In [ ]:
import sqlite3

con = sqlite3.connect("velociraptor_orgs")
db = con.cursor()

In [ ]:
# Create organization table
db.execute("CREATE TABLE IF NOT EXISTS  orgs('tenant_id','name','hash')")
db.execute("INSERT INTO orgs VALUES('root','root','4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2'),('OILK0','Eris','f08560478faa906ac431a301e8891358ce87358e9afeafe21ffc729521ead876'),('OV8LG','Nbs','0c5509358b879ece194a71004ef0c7f9edaf2c75d8c853d8c88c9f4d32dba729'),('OATOG','Nico','336442ecf0891552833cc9f63f1434f3f839b0b0671b2ec708182c5ad6a264c6')")

In [ ]:
# Select all tenants
db.execute("SELECT * FROM orgs;")
db.fetchall()

In [ ]:
con = sqlite3.connect("velociraptor_orgs")
dp = con.cursor()
dp.execute("DROP TABLE orgs;")

In [ ]:
# Close db connection
db.close()

In [ ]:
# JWT implementation
from jose import jwt
import datetime

# Set the access expiration time, it's set to expire in 1 year
access_expiration = datetime.datetime.now(datetime.UTC) + datetime.timedelta(hours=8760)

payload = {
    'exp': access_expiration,
    'api-client-id': hash,
}
algorithm='HS512'

In [ ]:
# JWT implementation
from jose import jwt
from Crypto.Hash import SHA256
import datetime
import sqlite3

# Registering API client to get API-Key
def register():
    id = "root"
    name = b"root"
    
    # Set the access expiration time, it's set to expire in 1 year
    access_expiration = datetime.datetime.now(datetime.UTC) + datetime.timedelta(hours=8760)
    hash = SHA256.new(name).hexdigest()

    payload = {
    'exp': access_expiration,
    'api-client-id': hash,
    }
    
    api_key = jwt.encode(payload,algorithm='HS512',key='4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2')
    api = api_key.split(".")[1:]
    api_key = ".".join(api) 
    print(id,api_key)
    con = sqlite3.connect("velociraptor_orgs")
    db = con.cursor()
    db.execute("INSERT INTO orgs VALUES(?,?)",(id,name))
    db.execute("INSERT INTO orgs_keys VALUES(?,?)",(id,api_key))
    db.close()

register()


In [14]:
import sqlite3

con = sqlite3.connect("velociraptor_orgs")
db = con.cursor()
db.execute("CREATE TABLE IF NOT EXISTS  orgs('id','name')")
da = con.cursor()
da.execute("CREATE TABLE IF NOT EXISTS  orgs_keys('id','api_key')")
db.close()
da.close()

In [16]:
import sqlite3
con = sqlite3.connect("velociraptor_orgs")
db = con.cursor()
db.execute("SELECT * FROM orgs_keys;")
data = db.fetchall()
db.close()
print(data)

[]


In [ ]:
def require_api_key():
        con = sqlite3.connect("velociraptor_orgs")
        db = con.cursor()
        db.execute("SELECT * FROM orgs_keys;")
        keys = db.fetchall()
        api_key = "eyJleHAiOjE3NTA5MjE3NDYsImFwaS1jbGllbnQtaWQiOiI0ODEzNDk0ZDEzN2UxNjMxYmJhMzAxZDVhY2FiNmU3YmI3YWE3NGNlMTE4NWQ0NTY1NjVlZjUxZDczNzY3N2IyIn0.-xNiUqg61eUKTEtQ_3TPPFJSLVbR8RT0Zqf0oq_YYw3dAgv_wXN3vYSmtA6UF0c97jkfqNK0KYEhQ_NpgaFvgQ"
        found = False

        for key in keys:
            if key[1] == api_key:
                found = True

        header = "eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9"
        token = header + "." + api_key

        res = jwt.decode(token)
        print(res)
require_api_key()

In [ ]:
def require_api_key(view_func):
    def decorated_function(*args, **kwargs):
        con = sqlite3.connect("velociraptor_orgs")
        db = con.cursor()
        db.execute("SELECT * FROM orgs_keys;")
        keys = db.fetchall()
        api_key = request.headers.get('Authorization')
        found = False

        for key in keys:
            if key[1] == api_key:
                found = True

        header = "eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9"
        token = header + "." + api_key

        res = jwt.decode(token)
        print(res)
        if request.headers.get('Authorization') == API_KEY:
            return view_func(*args, **kwargs)
        else:
            return jsonify(message='Unauthorized access'), 401

    return decorated_function